In [335]:
from sklearn.model_selection import train_test_split
from pandas import DataFrame, read_csv, concat, get_dummies
from sklearn import metrics
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.preprocessing import normalize, StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
import xgboost
from xgboost import plot_importance, XGBRegressor

import numpy as np
from math import sqrt

%matplotlib inline

In [336]:
project = 'house-prices'
version='v0.1'

In [337]:
fulltrain=read_csv('./train.csv',index_col=0)
test=read_csv('./test.csv',index_col=0)
fixed_seed=1234578416
train80, valid20 = train_test_split(fulltrain, test_size=0.2, random_state=fixed_seed)

In [338]:

#train80_qual = train80.select_dtypes(exclude=np.number)
#valid20_qual = valid20.select_dtypes(exclude=np.number)
#test_qual = test.select_dtypes(exclude=np.number)
#train80 = train80.select_dtypes(include=np.number)
#valid20 = valid20.select_dtypes(include=np.number)
#test = test.select_dtypes(include=np.number)

useless_fields=['MoSold', 'YrSold','Utilities']

train80.drop(useless_fields, axis=1, inplace=True)
valid20.drop(useless_fields, axis=1, inplace=True)
test.drop(useless_fields, axis=1, inplace=True)

qual_columns = train80.select_dtypes(exclude=np.number).columns

/home/guillaume/.virtualenvs/House-Prices/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [339]:
qual_columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType',
       'SaleCondition'],
      dtype='object')

In [340]:
train80.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'O

Remove outliers

In [341]:
train80 = train80.drop(train80[train80.LotArea > 200000].index)
#valid20 = valid20.drop(valid20[valid20.LotArea > 200000].index)

train80 = train80.drop(train80[train80.LotFrontage > 250].index)
#valid20 = valid20.drop(valid20[valid20.LotFrontage > 250].index)

In [342]:
train80.loc[:,'SalePrice']=np.log(train80.loc[:,'SalePrice'])
valid20.loc[:,'SalePrice']=np.log(valid20.loc[:,'SalePrice'])

/home/guillaume/.virtualenvs/House-Prices/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [343]:
mean_price_neigh={}
for name in list(train80.Neighborhood.unique()):
    mean_price_neigh[name]=train80.loc[train80['Neighborhood']==name,'SalePrice'].mean()
    
for name in list(train80.Neighborhood.unique()):
    train80.loc[train80['Neighborhood']==name,'mean_price']=mean_price_neigh[name]
    valid20.loc[valid20['Neighborhood']==name,'mean_price']=mean_price_neigh[name]
    test.loc[test['Neighborhood']==name,'mean_price']=mean_price_neigh[name]

# train80_qual['Neighborhood']
#mean_price_neigh

/home/guillaume/.virtualenvs/House-Prices/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [344]:
mean = {}
mode = {}
median ={}
for column in list(train80.columns):
    if column not in qual_columns:
        mean[column]=train80[column].mean()
        mode[column]=train80[column].mode()
        median[column]=train80[column].median()

In [345]:
target_column='SalePrice'
X_train = train80.drop(target_column, axis=1)
y_train = train80[target_column]
X_val = valid20.drop(target_column, axis=1)
y_val = valid20[target_column]

In [346]:
for df in [X_train, X_val, test]:
    df.loc[:,'Exterior1st'] = df['Exterior1st'].fillna('VinylSd')
    df.loc[:,'Exterior2nd'] = df['Exterior2nd'].fillna('VinylSd')
    df.loc[:,'KitchenQual'] = df['KitchenQual'].fillna('TA')
    df.loc[:,'Functional'] = df['Functional'].fillna('Typ')
    df.loc[:,'MSZoning'] = df['MSZoning'].fillna('RL')
    df.loc[:,'SaleType'] = df['SaleType'].fillna('WD')
    for col in ('PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'MasVnrType'):
        df.loc[:,col] = df[col].fillna('None')
    
    for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'GarageType'):
        df.loc[:,col] = df[col].fillna('None')
    
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea'):
        df.loc[:,col] = df[col].fillna(0)
    
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        df.loc[:,col] = df[col].fillna('None')    

    for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
        df.loc[:,col] = df[col].fillna(0)
        
    df.loc[:,'Electrical'] = df['Electrical'].fillna('SBrkr')
    
    # df = df.fillna(0)

In [347]:
#X_train = get_dummies(X_train, columns=qual_columns, drop_first=True)
#X_val = get_dummies(X_val, columns=qual_columns, drop_first=True)
#test = get_dummies(test, columns=qual_columns, drop_first=True)



In [348]:
X_train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'O

In [349]:
full = concat([X_train, X_val, test])
for col in qual_columns:
    # print(col)
    lbl = LabelEncoder()
    values = full[col].values
    lbl.fit(values)
    for df in [X_train, X_val, test]:
        df[col]=lbl.transform(df[col].values)
        

In [350]:
full = concat([X_train, X_val, test])
id_col=[]
for i in range(X_train.shape[1]):
    if X_train.columns[i] in qual_columns:
        id_col.append(i)
print(id_col)

#ord_enc = OrdinalEncoder()
#ord_enc.fit(full)
#X_train = ord_enc.transform(X_train)
#X_val = ord_enc.transform(X_val)
#test = ord_enc.transform(test)
#
#OHE = OneHotEncoder(categorical_features=id_col)
#OHE.fit(full)
#X_train = OHE.transform(X_train)
#X_val = OHE.transform(X_val)
#test = OHE.transform(test)

[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 33, 37, 38, 39, 40, 51, 53, 55, 56, 58, 61, 62, 63, 70, 71, 72, 74, 75]


Surface Area

In [351]:
for df in [X_train, X_val, test]:
    # df['Surface1'] =  df['2ndFlrSF'] * 0.67918 + df['1stFlrSF'] * 0.15835 + df['TotalBsmtSF'] * 0.56624
    # df['Surface2'] =  df['1stFlrSF'] * 0.51 + df['TotalBsmtSF'] * 0.35
    df['Surface'] =  df['2ndFlrSF'] + df['1stFlrSF'] + df['TotalBsmtSF']
#    df.drop('2ndFlrSF', axis=1, inplace=True)
#    df.drop('1stFlrSF', axis=1, inplace=True)
#    df.drop('TotalBsmtSF', axis=1, inplace=True)
    df.drop('GrLivArea', axis=1, inplace=True)
#    df['Garage'] = df['GarageCond'] + df['GarageFinish'] + df['GarageQual'] + df['GarageCars']

In [352]:
# normalization
scaler = StandardScaler()
scaler.fit(X_train)

X_train[:] = scaler.transform(X_train)
X_val[:] = scaler.transform(X_val)
test[:] = scaler.transform(test)

In [353]:
X_train.to_csv('X_train.csv')
X_val.to_csv('X_val.csv')
test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv', header=True)
y_val.to_csv('y_val.csv', header=True)
#, X_val, X_test
#y_train y_val

In [354]:
model = XGBRegressor(booster="gbtree",colsample_bytree=0.5,
 max_depth=3, n_estimators=400, subsample=0.7)

In [355]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=400,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=0.7)

In [356]:
y_pred = model.predict(X_val)

In [357]:
RMSLE = sqrt(mean_squared_error(y_val,y_pred))
RMSLE

0.12926601590184864

In [358]:
test_pred = model.predict(test)

In [359]:
submission = DataFrame({"SalePrice": np.exp (test_pred)}, index=test.index)
submission.to_csv('test-prediction.csv')